In [ ]:
# Completely remove both packages and cached wheels
!pip uninstall -y parselmouth praat-parselmouth googleads
!pip cache purge
!rm -rf /root/.cache/pip /usr/local/lib/python*/dist-packages/parselmouth* /usr/local/lib/python*/dist-packages/googleads*
!pip install praat-parselmouth==0.4.3 --no-cache-dir

Found existing installation: praat-parselmouth 0.4.3
Uninstalling praat-parselmouth-0.4.3:
  Successfully uninstalled praat-parselmouth-0.4.3
Files removed: 54 (3.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 23.6 MB/s  0:00:00


In [ ]:

!pip install --upgrade pip setuptools wheel


# Core dependencies
!pip install --upgrade librosa soundfile pandas scipy scikit-learn matplotlib tqdm torch torchvision transformers keras tensorflow

# Specialized scientific & signal libraries
!pip install PyWavelets
!pip install git+https://github.com/CSchoel/nolds.git
!pip install git+https://github.com/kymatio/kymatio.git






  Cloning https://github.com/CSchoel/nolds.git to /tmp/pip-req-build-a4gpo_w7
  Running command git clone --filter=blob:none --quiet https://github.com/CSchoel/nolds.git /tmp/pip-req-build-a4gpo_w7
  Resolved https://github.com/CSchoel/nolds.git to commit f121feebf6b2cbd67a8cb855bd0fe8c39add3ad9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/kymatio/kymatio.git to /tmp/pip-req-build-fyslrdl8
  Running command git clone --filter=blob:none --quiet https://github.com/kymatio/kymatio.git /tmp/pip-req-build-fyslrdl8
  Resolved https://github.com/kymatio/kymatio.git to commit e1fe488657c03a0a9c48e727b61fb5eea7005d82
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# ============================================
# 1. SETUP & INSTALLS
# ============================================
!pip install librosa soundfile pandas scipy scikit-learn matplotlib tqdm torch torchvision transformers keras tensorflow pywavelets nolds parselmouth kymatio joblib --quiet

import os, numpy as np, pandas as pd, librosa, torch, pywt, nolds, joblib
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from kymatio import Scattering1D
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, LSTM, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import parselmouth
from parselmouth.praat import call as praat_call
from collections import Counter

# ============================================
# 2. DEVICE & WAV2VEC2 LOADING
# ============================================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", DEVICE)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)

# ============================================
# 3. FEATURE EXTRACTION FUNCTIONS
# ============================================
def load_trim(path, sr=16000):
    y, s = librosa.load(path, sr=sr)
    y, _ = librosa.effects.trim(y, top_db=30)
    return y, s

def classical_feats(path):
    try:
        snd = parselmouth.Sound(path)
        pp = praat_call(snd, "To PointProcess (periodic, cc)", 75, 500)
        jitter = praat_call(pp, "Get jitter (local)", 0, 0, 75, 500, 1.3)
        shimmer = praat_call([snd, pp], "Get shimmer (local)", 0, 0, 75, 500, 1.3, 1.6)
        hnr = praat_call(snd, "Get harmonics-to-noise ratio", 0, 0)
        return {"jitter": jitter, "shimmer": shimmer, "hnr": hnr}
    except Exception:
        return {"jitter": np.nan, "shimmer": np.nan, "hnr": np.nan}

def nonlinear_feats(y):
    def perm_entropy(sig):
        order, delay = 3, 1
        patterns, n = {}, len(sig)
        for i in range(n - (order - 1) * delay):
            ranks = tuple(np.argsort(sig[i:i + order * delay:delay]))
            patterns[ranks] = patterns.get(ranks, 0) + 1
        ps = np.array(list(patterns.values()), dtype=float)
        ps /= ps.sum() + 1e-12
        return -np.sum(ps * np.log2(ps + 1e-12))
    try:
        y_ds = librosa.resample(y, orig_sr=16000, target_sr=1000)
        return {
            "hurst": nolds.hurst_rs(y_ds),
            "lyapunov": nolds.lyap_r(y_ds),
            "perm_entropy": perm_entropy(y_ds)
        }
    except Exception:
        return {"hurst": np.nan, "lyapunov": np.nan, "perm_entropy": np.nan}

def wavelet_feats(y):
    try:
        coeffs = pywt.wavedec(y, 'db4', level=5)
        energy = [np.sum(c**2) for c in coeffs]
        total = sum(energy) + 1e-12
        return {f"wavelet_{i}_ratio": energy[i] / total for i in range(len(energy))}
    except Exception:
        return {f"wavelet_{i}_ratio": np.nan for i in range(6)}

def scattering_feats(y):
    try:
        N = 2 ** int(np.ceil(np.log2(len(y))))
        y_pad = np.zeros(N)
        y_pad[:len(y)] = y
        scatter = Scattering1D(J=6, shape=N, Q=8)
        Sx = scatter(torch.from_numpy(y_pad).unsqueeze(0).float())
        Sx = Sx.squeeze(0).numpy() if hasattr(Sx, "numpy") else np.squeeze(Sx, axis=0)
        return {f"scat_mean_{i}": float(np.mean(Sx[i])) for i in range(min(30, Sx.shape[0]))}
    except Exception:
        return {f"scat_mean_{i}": np.nan for i in range(30)}

def wav2vec2_feats(y, sr):
    try:
        inputs = processor(y, sampling_rate=sr, return_tensors="pt", padding=True)
        with torch.no_grad():
            emb = wav2vec_model(inputs.input_values.to(DEVICE)).last_hidden_state.squeeze(0).cpu().numpy()
        return {"wav2vec_mean": np.mean(emb), "wav2vec_std": np.std(emb)}
    except Exception:
        return {"wav2vec_mean": np.nan, "wav2vec_std": np.nan}

# ============================================
# 4. LOAD DATASET
# ============================================
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset"
samples = [os.path.join(r, f) for r, _, fs in os.walk(DATASET_PATH) for f in fs if f.lower().endswith(".wav")]
print(f"Found {len(samples)} .wav files\n")

# ✅ Print first 30 file paths for inspection
print("📂 First 30 files in dataset:")
for f in samples[:30]:
    print(f)

# ============================================
# 4A. DEBUGGING: FILE EXISTENCE CHECK
# ============================================
missing_files = [f for f in samples if not os.path.exists(f)]
if missing_files:
    print(f"\n❌ Missing files detected ({len(missing_files)}):")
    print(missing_files[:20])
else:
    print("\n✅ All listed files exist.")

# ============================================
# 4B. DEBUGGING: LABEL PREVIEW
# ============================================
folder_names = [os.path.basename(os.path.dirname(f)).lower() for f in samples]
print("\n📊 Folder distribution:")
print(Counter(folder_names))

# ============================================
# 5. FEATURE EXTRACTION PIPELINE
# ============================================
rows = []
for file in tqdm(samples, desc="Extracting features"):
    try:
        if not os.path.exists(file):
            print("Error: File missing:", file)
            continue
        y, sr = load_trim(file)
        feats = {}
        feats.update(classical_feats(file))
        feats.update(nonlinear_feats(y))
        feats.update(wavelet_feats(y))
        feats.update(scattering_feats(y))
        feats.update(wav2vec2_feats(y, sr))

        folder = os.path.basename(os.path.dirname(file)).lower()

        # ✅ UPDATED LABEL MAPPING
        if folder in ["wp1111", "ic1111"]:
            feats["label"] = "Parkinson"
        elif folder in ["dl", "lw", "tessi", "bg_au", "mj_au", "sk_au", "jc_au", "ts_au", "tp_au"]:
            feats["label"] = "Healthy"
        else:
            print(f"⚠️ Unknown label for file: {file}")
            continue

        feats["filename"] = os.path.basename(file)
        rows.append(feats)
    except Exception as e:
        print("Error:", file, e)

df = pd.DataFrame(rows)
if df.empty:
    raise ValueError("❌ No valid samples extracted! Check label mapping and file paths.")
else:
    df.to_csv("full_features_combined.csv", index=False)
    print("\n✅ Feature extraction complete. Saved as full_features_combined.csv")
    print("\n📊 Label Distribution:")
    print(df["label"].value_counts())

# ============================================
# 6. PREPROCESSING: IMPUTE + SCALE + PCA
# ============================================
X = df.drop(["filename", "label"], axis=1)
y = df["label"]

drop_cols = X.columns[X.isna().all()].tolist()
if drop_cols:
    print(f"\nDropping columns with ALL NaN values: {drop_cols}")
    X = X.drop(columns=drop_cols)

le = LabelEncoder()
y = le.fit_transform(y)
label_mapping = dict(zip(le.transform(le.classes_), le.classes_))
print("\nLabel mapping:", label_mapping)
print("Label counts:", dict(pd.Series(y).value_counts()))

if len(np.unique(y)) < 2:
    raise ValueError("❌ Only one class found. Check file naming or label extraction.")

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95))
])
X_pca = pipeline.fit_transform(X)
joblib.dump(pipeline, "preprocessing_pipeline.joblib")
print("✅ PCA completed. Preprocessing pipeline saved.")

# ============================================
# 7. CLASSICAL ML MODELS
# ============================================
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)

models = {
    "Logistic": LogisticRegression(max_iter=1000),
    "SVM": SVC(kernel='rbf', probability=True),
    "RandomForest": RandomForestClassifier(200),
    "GradientBoosting": GradientBoostingClassifier(),
    "ExtraTrees": ExtraTreesClassifier()
}

best_model_name, best_model, best_acc = None, None, 0.0

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"\n{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, preds, target_names=le.classes_))
    if acc > best_acc:
        best_acc, best_model_name, best_model = acc, name, model

joblib.dump(best_model, f"best_ML_model_{best_model_name}.joblib")
print(f"\n🏆 Best ML Model: {best_model_name} ({best_acc:.4f}) saved successfully!")

# ============================================
# 8. DEEP LEARNING MODELS (CNN / LSTM / MLP)
# ============================================
X_dl = np.expand_dims(X_pca, axis=2)
y_dl = tf.keras.utils.to_categorical(y)
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_dl, y_dl, test_size=0.2, random_state=42)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

def build_and_train(model, name, X_train, y_train, X_test, y_test):
    checkpoint = ModelCheckpoint(f"best_DL_model_{name}.h5", monitor='val_accuracy', save_best_only=True, mode='max')
    model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stop, checkpoint], verbose=1)
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"{name} Test Accuracy: {acc:.4f}")
    return acc

# CNN
cnn = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_dl.shape[1], 1)),
    MaxPooling1D(2),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(y_dl.shape[1], activation='softmax')
])
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_acc = build_and_train(cnn, "CNN", X_train_dl, y_train_dl, X_test_dl, y_test_dl)

# LSTM
lstm = Sequential([
    LSTM(64, input_shape=(X_dl.shape[1], 1), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(y_dl.shape[1], activation='softmax')
])
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_acc = build_and_train(lstm, "LSTM", X_train_dl, y_train_dl, X_test_dl, y_test_dl)

# MLP
mlp = Sequential([
    Dense(256, activation='relu', input_shape=(X_dl.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_dl.shape[1], activation='softmax')
])
mlp.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mlp_acc = build_and_train(
    mlp, "MLP",
    X_train_dl.reshape(len(X_train_dl), -1), y_train_dl,
    X_test_dl.reshape(len(X_test_dl), -1), y_test_dl
)

best_dl_acc = max(cnn_acc, lstm_acc, mlp_acc)
print(f"\n🏆 Best Deep Learning Model Accuracy: {best_dl_acc:.4f}")
print("✅ All best models and preprocessing pipeline saved successfully.")







  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
ERROR: Failed to build 'googleads' when getting requirements to build wheel
Running on: cpu


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 578 .wav files

📂 First 30 files in dataset:
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW20.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW13.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW10.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW15.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW1.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW14.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW21.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW18.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW6.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW2.wav
/content/drive/MyDrive/Colab Notebooks/denoised-speech-dataset/LW/LW12.wa

Extracting features: 100%|██████████| 578/578 [25:28<00:00,  2.64s/it]



✅ Feature extraction complete. Saved as full_features_combined.csv

📊 Label Distribution:
label
Parkinson    361
Healthy      217
Name: count, dtype: int64

Dropping columns with ALL NaN values: ['jitter', 'shimmer', 'hnr']

Label mapping: {np.int64(0): 'Healthy', np.int64(1): 'Parkinson'}
Label counts: {1: np.int64(361), 0: np.int64(217)}
✅ PCA completed. Preprocessing pipeline saved.

Logistic Accuracy: 0.9828
              precision    recall  f1-score   support

     Healthy       0.98      0.98      0.98        44
   Parkinson       0.99      0.99      0.99        72

    accuracy                           0.98       116
   macro avg       0.98      0.98      0.98       116
weighted avg       0.98      0.98      0.98       116


SVM Accuracy: 0.9569
              precision    recall  f1-score   support

     Healthy       0.91      0.98      0.95        44
   Parkinson       0.99      0.94      0.96        72

    accuracy                           0.96       116
   macro avg    

...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 1/12 ━━━━━━━━━━━━━━━━━━━━ 9s 849ms/step - accuracy: 0.5625 - loss: 0.6432

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7724 - loss: 0.4867 - val_accuracy: 0.9032 - val_loss: 0.2516
Epoch 2/30
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8438 - loss: 0.4973

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9079 - loss: 0.2830 - val_accuracy: 0.9677 - val_loss: 0.1626
Epoch 3/30
 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8750 - loss: 0.2752

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9295 - loss: 0.2039 - val_accuracy: 0.9785 - val_loss: 0.1257
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9648 - loss: 0.1474 - val_accuracy: 0.9677 - val_loss: 0.1128
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9593 - loss: 0.1202 - val_accuracy: 0.9677 - val_loss: 0.1019
MLP Test Accuracy: 0.8190

🏆 Best Deep Learning Model Accuracy: 0.9655
✅ All best models and preprocessing pipeline saved successfully.
